In [1]:
print(5)

5


In [13]:
from io import StringIO
import requests
import json
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
url = "https://query1.finance.yahoo.com/v7/finance/download/BTC-USD?period1=0000000000&period2=2000000000&interval=1d&events=history&includeAdjustedClose=true"
response = requests.get(url)
response.text
df = pd.read_csv(StringIO(response.text),index_col = "Date",parse_dates = ["Date"])
df_plot = df[{'Open','High','Low','Close'}]

In [15]:
df_plot.tail()

,High,Low,Close,Open
Date,,,,
2021-02-28,46716.429688,43241.617188,45137.769531,46194.015625
2021-03-01,49784.015625,45115.093750,49631.242188,45159.503906
2021-03-02,50127.511719,47228.843750,48378.988281,49612.105469
2021-03-03,52535.136719,48274.320313,50538.242188,48415.816406
2021-03-04,51735.089844,47656.929688,48561.167969,50522.304688


In [4]:
#計算ma
def moving_average(data,periods):
    return data['Close'].rolling(periods).mean()

In [5]:
df_copy = df.copy()
df_copy['min'] = df_copy['Low'].rolling(9).min()
df_copy['max'] = df_copy['High'].rolling(9).min()
df_copy['RSV'] = (df_copy['Close'] - df_copy['min']) / (df_copy['max'] - df_copy['min'])
df_copy = df_copy.dropna()
K_list = [50]
for index,rsv in enumerate(list(df_copy['RSV'])):
    K_yesterday = K_list[index]
    K_today = 2/3 * K_yesterday + 1/3 *rsv
    K_list.append(K_today)
df_copy['K'] = K_list[1:]
D_list = [50]
for index,K in enumerate(list(df_copy['K'])):
    D_yesterday = D_list[index]
    D_today = 2/3 * D_yesterday + 1/3 *K
    D_list.append(D_today)
df_copy['D'] = D_list[1:]

In [6]:
df_copy.tail()

,Open,High,Low,Close,Adj Close,Volume,min,max,RSV,K,D
Date,,,,,,,,,,,
2021-02-28,46194.015625,46716.429688,43241.617188,45137.769531,45137.769531,5.344389e+10,43241.617188,46716.429688,0.545685,0.860156,1.540121
2021-03-01,45159.503906,49784.015625,45115.093750,49631.242188,49631.242188,5.389130e+10,43241.617188,46716.429688,1.838840,1.186384,1.422208
2021-03-02,49612.105469,50127.511719,47228.843750,48378.988281,48378.988281,4.753090e+10,43241.617188,46716.429688,1.478460,1.283743,1.376053
2021-03-03,48415.816406,52535.136719,48274.320313,50538.242188,50538.242188,5.322081e+10,43241.617188,46716.429688,2.099862,1.555782,1.435963
2021-03-04,50522.304688,51735.089844,47656.929688,48561.167969,48561.167969,5.234382e+10,43241.617188,46716.429688,1.530889,1.547484,1.473137


In [7]:
#計算KD值
'''
step1 計算RSV:(今天的收盤價 - 最近9天的最低價)/(最近9天的最高價 - 最近9天的最低價)
step2 計算K值: K = 2/3 * (昨天的K值) + 1/3 * (今日RSV)
step3 計算D值: K = 2/3 * (昨天的D值) + 1/3 * (今日的K值)
'''
def KD(data):
    df_copy = data.copy()
    df_copy['min'] = df_copy['Low'].rolling(9).min()
    df_copy['max'] = df_copy['High'].rolling(9).min()
    df_copy['RSV'] = (df_copy['Close'] - df_copy['min']) / (df_copy['max'] - df_copy['min'])
    df_copy = df_copy.dropna()
    K_list = [50]
    for index,rsv in enumerate(list(df_copy['RSV'])):
        K_yesterday = K_list[index]
        K_today = 2/3 * K_yesterday + 1/3 *rsv
        K_list.append(K_today)
    df_copy['K'] = K_list[1:]
    D_list = [50]
    for index,K in enumerate(list(df_copy['K'])):
        D_yesterday = D_list[index]
        D_today = 2/3 * D_yesterday + 1/3 *K
        D_list.append(D_today)
    df_copy['D'] = D_list[1:]
    use_df = pd.merge(data,df_copy[['K','D']],left_index=True,right_index=True,how='left')
    return use_df

In [8]:
KD(df).tail(10)

,Open,High,Low,Close,Adj Close,Volume,K,D
Date,,,,,,,,
2021-02-23,54204.929688,54204.929688,45290.589844,48824.425781,48824.425781,1.061025e+11,3.176372,3.856953
2021-02-24,48835.085938,51290.136719,47213.500000,49705.332031,49705.332031,6.369552e+10,2.408954,3.374286
2021-02-25,49709.082031,51948.968750,47093.851563,47093.851563,47093.851563,5.450657e+10,1.706158,2.818244
2021-02-26,47180.464844,48370.785156,44454.843750,46339.761719,46339.761719,3.509679e+11,1.297887,2.311458
2021-02-27,46344.773438,48253.269531,45269.027344,46188.453125,46188.453125,4.591095e+10,1.017392,1.880103
2021-02-28,46194.015625,46716.429688,43241.617188,45137.769531,45137.769531,5.344389e+10,0.860156,1.540121
2021-03-01,45159.503906,49784.015625,45115.093750,49631.242188,49631.242188,5.389130e+10,1.186384,1.422208
2021-03-02,49612.105469,50127.511719,47228.843750,48378.988281,48378.988281,4.753090e+10,1.283743,1.376053
2021-03-03,48415.816406,52535.136719,48274.320313,50538.242188,50538.242188,5.322081e+10,1.555782,1.435963
